# Setup

Instalar dependências

In [ ]:
!pip install pyxlsb

Importar dependências

In [ ]:
import os
import random
import gc
from glob import glob
import zipfile

import pandas as pd
from pyxlsb import open_workbook

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from google.colab import drive


(Colab) Conectar com o google drive

In [ ]:
drive.mount('/content/drive')

Variáveis de ambiente

In [ ]:
DATA_PATH = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'

# Pré-processamento de dados

## **Carregamento dos Caminhos de Imagem**

Carregar os caminhos das imagens e máscaras.


In [ ]:
images = []
masks = []

for path in glob(DATA_PATH):
    images.append(path + '/image.tif')
    masks.append(path + '/mask.png')

print("Caminhos das máscaras carregados:", masks)

## **Pré-processamento de Imagens**
Carrega e pré-processa uma imagem e sua máscara correspondente.


In [ ]:
def load_and_preprocess_image(image_path, mask_path, target_size=(256, 256)):
    """
    Carrega e pré-processa uma imagem e sua máscara correspondente.
    Parâmetros:
        image_path (str): Caminho para a imagem.
        mask_path (str): Caminho para a máscara.
        target_size (tuple): Dimensões para redimensionamento das imagens.
    Retorna:
        tuple: Uma tupla contendo a imagem e a máscara processadas.
    """
    # Carregamento e normalização da imagem
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0

    # Carregamento e normalização da máscara
    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0

    return image, mask

## **Processamento em Lote e Preparação dos Dados**
Divisão em lotes para facilitar o processamento


In [ ]:
images_processed = []
masks_processed = []
count = 1

for img_path, mask_path in zip(images, masks):
    print(f'Processando imagem {count}')
    img, mask = load_and_preprocess_image(img_path, mask_path)
    images_processed.append(img)
    masks_processed.append(mask)
    count += 1

# Conversão para arrays numpy para facilitar o processamento no TensorFlow/Keras
images_processed = np.array(images_processed)
masks_processed = np.array(masks_processed)

## **Divisão dos Dados para Treinamento e Validação**

Prepara os dados processados para utilização no modelo

In [ ]:
try:
    X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.2, random_state=42)
except Exception as e:
    print(f'Erro ao dividir os dados: {e}')

# Definição dos modelos

## **Definição do Modelo U-Net com Encoder MobileNetV2**


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers

def Unet_with_MobileNetV2_encoder(input_shape=(256, 256, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=inputs)
    base_model.trainable = False

    # Intermediate layers for skip connections
    skip1 = base_model.get_layer('block_1_expand_relu').output
    skip2 = base_model.get_layer('block_3_expand_relu').output
    encoder_output = base_model.get_layer('block_6_expand_relu').output

    # Decoder
    up1 = layers.Conv2DTranspose(96, (2, 2), strides=(2, 2), padding='same')(encoder_output)
    merge1 = layers.concatenate([skip2, up1], axis=3)
    conv1 = layers.Conv2D(96, 3, activation='relu', padding='same')(merge1)
    conv1 = layers.Conv2D(96, 3, activation='relu', padding='same')(conv1)

    up2 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv1)
    merge2 = layers.concatenate([skip1, up2], axis=3)
    conv2 = layers.Conv2D(32, 3, activation='relu', padding='same')(merge2)
    conv2 = layers.Conv2D(32, 3, activation='relu', padding='same')(conv2)

    up3 = layers.Conv2DTranspose(24, (2, 2), strides=(2, 2), padding='same')(conv2)
    conv3 = layers.Conv2D(24, 3, activation='relu', padding='same')(up3)
    conv3 = layers.Conv2D(24, 3, activation='relu', padding='same')(conv3)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv3)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

## **Definição do Modelo SegNet com Encoder VGG16**

In [ ]:
def SegNet_with_VGG_encoder(input_shape=(256, 256, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=inputs)
    vgg16.trainable = False

    # Layer outputs for skip connections
    s1 = vgg16.get_layer('block1_pool').output
    s2 = vgg16.get_layer('block2_pool').output
    s3 = vgg16.get_layer('block3_pool').output
    s4 = vgg16.get_layer('block4_pool').output
    encoded = vgg16.get_layer('block5_pool').output

    # Decoder using Conv2DTranspose
    up1 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(encoded)
    up1 = layers.BatchNormalization()(up1)
    up1 = layers.Activation('relu')(up1)
    merge1 = layers.concatenate([s4, up1], axis=3)

    up2 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(merge1)
    up2 = layers.BatchNormalization()(up2)
    up2 = layers.Activation('relu')(up2)
    merge2 = layers.concatenate([s3, up2], axis=3)

    up3 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(merge2)
    up3 = layers.BatchNormalization()(up3)
    up3 = layers.Activation('relu')(up3)
    merge3 = layers.concatenate([s2, up3], axis=3)

    up4 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(merge3)
    up4 = layers.BatchNormalization()(up4)
    up4 = layers.Activation('relu')(up4)
    merge4 = layers.concatenate([s1, up4], axis=3)

    up5 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(merge4)
    up5 = layers.BatchNormalization()(up5)
    up5 = layers.Activation('relu')(up5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(up5)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Treino do modelo

## **Função para Criação e Treinamento de Modelos**

Cria, compila, treina um modelo de rede neural e avalia no conjunto de validação.

In [ ]:
def train_and_evaluate_model(model_function, input_shape, X_train, y_train, X_val, y_val):
    """
    Cria, compila, treina um modelo de rede neural e avalia no conjunto de validação.
    Parâmetros:
        model_function (function): Função que retorna um modelo compilado.
        input_shape (tuple): Dimensões da entrada do modelo.
        X_train, y_train: Dados de treinamento.
        X_val, y_val: Dados de validação.
    Retorna:
        dict: Histórico de treinamento do modelo e resultados de avaliação.
    """

    # Criar e compilar o modelo
    print('Criando e compilando o modelo...')
    model = model_function(input_shape)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Treinar o modelo
    print('Treinando o modelo...')
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, batch_size=32)

    # Prever máscaras usando o modelo
    print('Prevendo máscaras...')
    predicted_masks = model.predict(X_val)

    # Limpar a memória
    print('Processo finalizado. Resultados armazenados e limpando memória')
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    # Retornar o histórico de treinamento e os resultados de avaliação
    return history.history, predicted_masks

## **Treinamento Sequencial de Múltiplos Modelos**

Como estamos utilizando múltiplos modelos, iremos treiná-los sequencialmente.

In [ ]:
# Definir os parâmetros de entrada
input_shape = (256, 256, 3)

# Treinar o modelo U-Net com MobileNetV2
history_unet, predicted_masks_unet = train_and_evaluate_model(Unet_with_MobileNetV2_encoder, input_shape, X_train, y_train, X_val, y_val)

# Treinar o modelo SegNet com VGG
history_segnet, predicted_masks_segnet = train_and_evaluate_model(SegNet_with_VGG_encoder, input_shape, X_train, y_train, X_val, y_val)

# Exibir ou salvar os históricos para análise
print("Histórico de treinamento U-Net com MobileNetV2:", history_unet)
print("Histórico de treinamento SegNet com VGG:", history_segnet)

# Avaliação do modelo

## Visualização das máscaras

In [ ]:
# Visualizar comparações
for i in range(len(X_val)):
    img_entrada = X_val[i]
    img_saida_real = y_val[i]
    img_saida_unet = np.where(predicted_masks_unet[i] < 0.5, 0, 1)
    img_saida_segnet = np.where(predicted_masks_segnet[i] < 0.5, 0, 1)

    plt.figure(figsize=(16, 4))

    plt.subplot(1, 5, 1)
    plt.imshow(img_entrada)
    plt.title('Entrada')
    plt.axis('off')

    plt.subplot(1, 5, 2)
    plt.imshow(img_saida_real.squeeze(), cmap='gray')
    plt.title('Saída Esperada')
    plt.axis('off')

    plt.subplot(1, 5, 3)
    plt.imshow(img_saida_unet.squeeze(), cmap='gray')
    plt.title('Saída U-Net')
    plt.axis('off')

    plt.subplot(1, 5, 4)
    plt.imshow(img_saida_segnet.squeeze(), cmap='gray')
    plt.title('Saída SegNet')
    plt.axis('off')

    plt.show()

## Comparação de métricas

In [ ]:
import matplotlib.pyplot as plt

# Dados do histórico para o U-Net
acc_unet = history_unet['accuracy']
val_acc_unet = history_unet['val_accuracy']
loss_unet = history_unet['loss']
val_loss_unet = history_unet['val_loss']

# Dados do histórico para o SegNet
acc_segnet = history_segnet['accuracy']
val_acc_segnet = history_segnet['val_accuracy']
loss_segnet = history_segnet['loss']
val_loss_segnet = history_segnet['val_loss']

# Número de épocas baseado nos dados do U-Net (assumindo mesmo número de épocas para ambos)
epochs = range(1, len(acc_unet) + 1)

# Criar figura para subplots
plt.figure(figsize=(14, 5))

# Subplot para precisão de U-Net
plt.subplot(1, 2, 1)
plt.plot(epochs, acc_unet, 'r', label='Precisão do Conjunto de Treino (U-Net)')
plt.plot(epochs, val_acc_unet, 'b', label='Precisão do Conjunto de Validação (U-Net)')
plt.title('Precisão do Treino e Validação - U-Net')
plt.xlabel('Épocas')
plt.ylabel('Precisão')
plt.legend()

# Subplot para precisão de SegNet
plt.subplot(1, 2, 2)
plt.plot(epochs, acc_segnet, 'r', label='Precisão do Conjunto de Treino (SegNet)')
plt.plot(epochs, val_acc_segnet, 'b', label='Precisão do Conjunto de Validação (SegNet)')
plt.title('Precisão do Treino e Validação - SegNet')
plt.xlabel('Épocas')
plt.ylabel('Precisão')
plt.legend()

# Mostrar os gráficos de precisão
plt.show()

# Criar figura para subplots de perda
plt.figure(figsize=(14, 5))

# Subplot para perda de U-Net
plt.subplot(1, 2, 1)
plt.plot(epochs, loss_unet, 'r', label='Perda do Conjunto de Treino (U-Net)')
plt.plot(epochs, val_loss_unet, 'b', label='Perda do Conjunto de Validação (U-Net)')
plt.title('Perda do Treino e Validação - U-Net')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()

# Subplot para perda de SegNet
plt.subplot(1, 2, 2)
plt.plot(epochs, loss_segnet, 'r', label='Perda do Conjunto de Treino (SegNet)')
plt.plot(epochs, val_loss_segnet, 'b', label='Perda do Conjunto de Validação (SegNet)')
plt.title('Perda do Treino e Validação - SegNet')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()

# Mostrar os gráficos de perda
plt.show()


## Calculo de IoU e Covr

In [ ]:
import numpy as np

def calculate_iou_and_coverage(y_val, predicted_masks, threshold=0.5):
    """
    Calcula o IoU e o Coverage Ratio das máscaras previstas em relação às verdadeiras.
    """
    iou_scores = []
    correct_predictions = 0

    # Processar cada par de máscara verdadeira e prevista
    for true_mask, pred_mask in zip(y_val, predicted_masks):
        pred_mask = (pred_mask > threshold).astype(int)  # Aplicar limiarização

        intersection = np.logical_and(true_mask, pred_mask)
        union = np.logical_or(true_mask, pred_mask)
        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0

        iou_scores.append(iou_score)

        if iou_score >= threshold:
            correct_predictions += 1

    iou_mean = np.mean(iou_scores)
    coverage_ratio = correct_predictions / len(iou_scores) if len(iou_scores) > 0 else 0

    return iou_mean, coverage_ratio


In [ ]:
# Calcular IoU e Coverage Ratio
iou_mean_unet, coverage_ratio_unet = calculate_iou_and_coverage(y_val, predicted_masks_unet)
iou_mean_segnet, coverage_ratio_segnet = calculate_iou_and_coverage(y_val, predicted_masks_segnet)

# Plotar os resultados
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.bar(['U-Net', 'SegNet'], [iou_mean_unet, iou_mean_segnet], color=['red', 'blue'])
plt.title('Comparação de IoU')
plt.ylabel('IoU')

plt.subplot(1, 2, 2)
plt.bar(['U-Net', 'SegNet'], [coverage_ratio_unet, coverage_ratio_segnet], color=['red', 'blue'])
plt.title('Comparação de Coverage Ratio')
plt.ylabel('Coverage Ratio')

plt.show()

# Conclusão

Tendo as análises acima em mente, é possível afirmar que a melhor escolha dentre os dois é o U-Net; visto que suas perdas na validação compensam o "tradeoff" de um coverage ratio levemente menor e uma precisão menor em treino.

Ao inspecionar visualmente as saídas, também é possível notar a qualidade das imagens, especialmente quando comparadas as da Segnet.